importing necessary libraries


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# importing  model
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,BernoulliNB,MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# vectorization model
from sklearn.feature_extraction.text import TfidfVectorizer

# for balancing the imbalance dataset
# !pip install imbalanced-learn (to install the model in your ide )
from imblearn.over_sampling import SMOTE
from collections import Counter

# tensorflow to build to model
import tensorflow
from tensorflow import keras
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input
from keras.layers import  Embedding
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential

import warnings
warnings.filterwarnings('ignore')


In [2]:
# reading the dataset
df=pd.read_csv('new.csv')

In [3]:
df.head()

,Unnamed: 0,class,transformed_tweet,len_word,num_word,positive_score,negative_score,sentiment
0,0,2,woman complain cleaning house amp man always t...,55,9,0.000,0.238,0.000000
1,1,1,boy dats cold tyga dwn bad cuffin dat hoe 1st ...,51,11,0.000,0.259,-0.650000
2,2,1,ever fuck bitch start cry confused shit,39,7,0.000,0.891,-0.333333
3,3,1,look like tranny,16,3,0.556,0.000,0.000000
4,4,1,shit hear might true might faker bitch told ya...,52,10,0.163,0.430,0.075000


In [4]:
df.isnull().sum()

Unnamed: 0              0
class                   0
transformed_tweet    2329
len_word                0
num_word                0
positive_score          0
negative_score          0
sentiment               0
dtype: int64

In [5]:
# droping the null values
new_df=df.dropna()

In [6]:
new_df.isnull().sum()

Unnamed: 0           0
class                0
transformed_tweet    0
len_word             0
num_word             0
positive_score       0
negative_score       0
sentiment            0
dtype: int64

In [7]:
new_df.duplicated().sum()

0

In [8]:
new_df['class'].value_counts().index

Int64Index([1, 2], dtype='int64')

In [9]:
new_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [10]:
new_df.head()

,class,transformed_tweet,len_word,num_word,positive_score,negative_score,sentiment
0,2,woman complain cleaning house amp man always t...,55,9,0.000,0.238,0.000000
1,1,boy dats cold tyga dwn bad cuffin dat hoe 1st ...,51,11,0.000,0.259,-0.650000
2,1,ever fuck bitch start cry confused shit,39,7,0.000,0.891,-0.333333
3,1,look like tranny,16,3,0.556,0.000,0.000000
4,1,shit hear might true might faker bitch told ya...,52,10,0.163,0.430,0.075000


In [11]:
new_df['class'].replace({1:0},inplace=True)
new_df['class'].replace({2:1},inplace=True)

In [12]:
new_df.shape

(22454, 7)

In [13]:
X=new_df.drop(['class'],axis=1)
y=new_df['class']

In [14]:
# X_new = []
# for item in X.tolist():
#   X_new.extend(item)
# X_new

X.head()


,transformed_tweet,len_word,num_word,positive_score,negative_score,sentiment
0,woman complain cleaning house amp man always t...,55,9,0.000,0.238,0.000000
1,boy dats cold tyga dwn bad cuffin dat hoe 1st ...,51,11,0.000,0.259,-0.650000
2,ever fuck bitch start cry confused shit,39,7,0.000,0.891,-0.333333
3,look like tranny,16,3,0.556,0.000,0.000000
4,shit hear might true might faker bitch told ya...,52,10,0.163,0.430,0.075000


In [15]:
 # splitting the training and testing data for model building
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=25)

In [16]:
X_train_tf = X_train['transformed_tweet']
X_train_ext_feature = X_train.drop(['transformed_tweet'], axis=1)
mc=MinMaxScaler()
X_train_ext_feature=mc.fit_transform(X_train_ext_feature)
print(X_train_tf.head())
X_train_ext_feature=pd.DataFrame(X_train_ext_feature)
print(X_train_ext_feature.head())

14527    chava needs stay lane keep sending gyro hell d...
5689                     caraabastow cute fuzzy one hehehe
14269    man insecure cause make feel like got ta compe...
21221                       summer hoes turn winter wifeys
8485     classy rich family kids drugs tryina ghetto sh...
Name: transformed_tweet, dtype: object
          0         1      2      3       4
0  0.148601  0.182927  0.000  0.494  0.5000
1  0.055944  0.048780  0.429  0.000  0.7500
2  0.139860  0.182927  0.115  0.332  0.2500
3  0.050699  0.048780  0.000  0.000  0.5000
4  0.173077  0.182927  0.258  0.266  0.4875


In [17]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(17963, 6)
(4491, 6)
(17963,)
(4491,)


In [18]:
# initilizing the vectorizer model
tfid=TfidfVectorizer(max_features=1000)

In [19]:
X_tf=tfid.fit_transform(X_train_tf)

In [20]:
X_tf.shape

(17963, 1000)

In [21]:
X_tf=X_tf.toarray()

In [22]:
type(X_tf)

numpy.ndarray

In [23]:
X_ext_feat = X_train_ext_feature.values
X_ext_feat

array([[0.1486014 , 0.18292683, 0.        , 0.494     , 0.5       ],
       [0.05594406, 0.04878049, 0.429     , 0.        , 0.75      ],
       [0.13986014, 0.18292683, 0.115     , 0.332     , 0.25      ],
       ...,
       [0.09965035, 0.12195122, 0.165     , 0.485     , 0.321875  ],
       [0.04195804, 0.04878049, 0.        , 0.487     , 0.5       ],
       [0.03146853, 0.02439024, 0.        , 0.608     , 0.5       ]])

In [24]:
X_ext_feat.shape

(17963, 5)

In [25]:
combined_data = pd.concat([pd.DataFrame(X_tf),
                           pd.DataFrame( X_ext_feat)], axis=1,ignore_index=True)

In [26]:
combined_data

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,1000,1001,1002,1003,1004
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.148601,0.182927,0.000,0.494,0.500000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.055944,0.048780,0.429,0.000,0.750000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.139860,0.182927,0.115,0.332,0.250000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.050699,0.048780,0.000,0.000,0.500000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.173077,0.182927,0.258,0.266,0.487500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.206812,0.0,0.000000,0.0,0.139860,0.170732,0.198,0.125,0.900000
17959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.097902,0.097561,0.277,0.000,0.500000
17960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.099650,0.121951,0.165,0.485,0.321875
17961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.041958,0.048780,0.000,0.487,0.500000


In [27]:
X_test_tf=X_test['transformed_tweet']
X_test_ext_feat=X_test.drop(['transformed_tweet'],axis=1)
X_test_ext_feat=mc.fit_transform(X_test_ext_feat)
X_test_ext_feat=pd.DataFrame(X_test_ext_feat)
print(X_test_ext_feat.head())

          0         1         2      3      4
0  0.193548  0.163265  0.000000  0.000  0.500
1  0.108504  0.102041  0.000000  0.612  0.350
2  0.152493  0.142857  0.178005  0.000  0.500
3  0.258065  0.306122  0.310658  0.459  0.575
4  0.090909  0.122449  0.000000  0.597  0.275


In [28]:
new_X_test=tfid.transform(X_test_tf)
new_X_test=new_X_test.toarray()

In [29]:
new_X_test.shape,X_test_ext_feat.shape


((4491, 1000), (4491, 5))

In [30]:
pd.DataFrame(new_X_test)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
X_test_ext_feat.shape[0]

4491

In [32]:
# pd.DataFrame(X_test_ext_feat,index=[i for i in range(0,X_test_ext_feat.shape[0])])

In [33]:
combined_data_test = pd.concat([pd.DataFrame(new_X_test),
                           pd.DataFrame( X_test_ext_feat.values)], axis=1,ignore_index=True)

In [34]:
combined_data_test

,0,1,2,3,4,5,6,7,8,9,...,995,996,997,998,999,1000,1001,1002,1003,1004
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.193548,0.163265,0.000000,0.000,0.500000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.108504,0.102041,0.000000,0.612,0.350000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.152493,0.142857,0.178005,0.000,0.500000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.258065,0.306122,0.310658,0.459,0.575000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.090909,0.122449,0.000000,0.597,0.275000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.058651,0.040816,0.000000,0.000,0.500000
4487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.126100,0.142857,0.531746,0.259,0.800000
4488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.217009,0.244898,0.140590,0.378,0.400000
4489,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.052786,0.040816,0.000000,0.655,0.500000


In [35]:
# initilizing the ml model
gn=GaussianNB()
bn=BernoulliNB()
mn=MultinomialNB()

In [36]:
# using GaussianNB model
gn.fit(combined_data,y_train)
pred1=gn.predict(combined_data_test)
print(pred1)
print(accuracy_score(y_test,pred1))
print(confusion_matrix(y_test,pred1))
print(precision_score(y_test,pred1))
print(classification_report(y_test, pred1))

[1 0 0 ... 0 0 0]
0.642618570474282
[[2245 1542]
 [  63  641]]
0.293632615666514
              precision    recall  f1-score   support

           0       0.97      0.59      0.74      3787
           1       0.29      0.91      0.44       704

    accuracy                           0.64      4491
   macro avg       0.63      0.75      0.59      4491
weighted avg       0.87      0.64      0.69      4491



In [37]:
# using BernoulliNB model
bn.fit(combined_data,y_train)
pred2=bn.predict(combined_data_test)
print(pred2)
print(accuracy_score(y_test,pred2))
print(confusion_matrix(y_test,pred2))
print(precision_score(y_test,pred2))
print(classification_report(y_test, pred2))

[1 0 0 ... 0 0 0]
0.9231796927187709
[[3577  210]
 [ 135  569]]
0.730423620025674
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      3787
           1       0.73      0.81      0.77       704

    accuracy                           0.92      4491
   macro avg       0.85      0.88      0.86      4491
weighted avg       0.93      0.92      0.92      4491



In [38]:
# using the random forest classifier
rf=RandomForestClassifier(n_estimators=200)
rf.fit(combined_data,y_train)
pred11=rf.predict(combined_data_test)
print(pred11)
print(accuracy_score(y_test,pred11))
print(confusion_matrix(y_test,pred11))
print(precision_score(y_test,pred11))
print(classification_report(y_test, pred11))

[1 0 1 ... 0 0 0]
0.9296370518815409
[[3635  152]
 [ 164  540]]
0.7803468208092486
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      3787
           1       0.78      0.77      0.77       704

    accuracy                           0.93      4491
   macro avg       0.87      0.86      0.87      4491
weighted avg       0.93      0.93      0.93      4491



In [39]:
# balancing the data

In [40]:
Counter(y_train)

Counter({0: 15174, 1: 2789})

applying SMOTE

In [41]:
smote=SMOTE(sampling_strategy=0.3)

In [42]:
X_train_sm,y_train_sm=smote.fit_resample(combined_data,y_train)

In [43]:
Counter(y_train_sm)

Counter({0: 15174, 1: 4552})

In [44]:
# now using the model after balancing the dataset

In [45]:
gn1=GaussianNB()
bn1=BernoulliNB()
mn1=MultinomialNB()

In [46]:
# using the BernoulliNB model after balancing the data
bn1.fit(X_train_sm,y_train_sm)
pred2=bn1.predict(combined_data_test)
print(pred2)
print(accuracy_score(y_test,pred2))
print(confusion_matrix(y_test,pred2))
print(precision_score(y_test,pred2))
print(classification_report(y_test, pred2))

[1 0 0 ... 0 0 0]
0.9225116900467601
[[3546  241]
 [ 107  597]]
0.7124105011933174
              precision    recall  f1-score   support

           0       0.97      0.94      0.95      3787
           1       0.71      0.85      0.77       704

    accuracy                           0.92      4491
   macro avg       0.84      0.89      0.86      4491
weighted avg       0.93      0.92      0.93      4491



In [47]:
# using the GaussianNB model after balancing the data
gn1.fit(X_train_sm,y_train_sm)
pred0=gn1.predict(combined_data_test)
print(pred0)
print(accuracy_score(y_test,pred0))
print(confusion_matrix(y_test,pred0))
print(precision_score(y_test,pred0))
print(classification_report(y_test, pred0))

[1 0 0 ... 0 0 0]
0.6789133823201959
[[2436 1351]
 [  91  613]]
0.3121181262729124
              precision    recall  f1-score   support

           0       0.96      0.64      0.77      3787
           1       0.31      0.87      0.46       704

    accuracy                           0.68      4491
   macro avg       0.64      0.76      0.62      4491
weighted avg       0.86      0.68      0.72      4491



In [53]:
# using random forest classifier
rf1=RandomForestClassifier(n_estimators=200)
rf1.fit(X_train_sm,y_train_sm)
pred11=rf.predict(combined_data_test)
print(pred11)
print(accuracy_score(y_test,pred11))
print(confusion_matrix(y_test,pred11))
print(precision_score(y_test,pred11))
print(classification_report(y_test, pred11))

[1 0 1 ... 0 0 0]
0.9307503896682253
[[3600  187]
 [ 124  580]]
0.7561929595827901
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      3787
           1       0.76      0.82      0.79       704

    accuracy                           0.93      4491
   macro avg       0.86      0.89      0.87      4491
weighted avg       0.93      0.93      0.93      4491



In [49]:
# using the logistic regression
lr=LogisticRegression()
lr.fit(X_train_sm,y_train_sm)
pred_lr=lr.predict(combined_data_test)
print(classification_report(y_test, pred_lr))


              precision    recall  f1-score   support

           0       0.97      0.95      0.96      3787
           1       0.75      0.85      0.80       704

    accuracy                           0.93      4491
   macro avg       0.86      0.90      0.88      4491
weighted avg       0.94      0.93      0.93      4491



In [50]:
# using the XGBoost Classifier
xg = XGBClassifier()
xg.fit(X_train_sm, y_train_sm)
pred_xg=xg.predict(combined_data_test)
print(classification_report(y_test, pred_xg))

              precision    recall  f1-score   support

           0       0.98      0.94      0.96      3787
           1       0.73      0.90      0.80       704

    accuracy                           0.93      4491
   macro avg       0.85      0.92      0.88      4491
weighted avg       0.94      0.93      0.93      4491



In [55]:
print(confusion_matrix(y_test,pred_xg))

[[3551  236]
 [  73  631]]


In [51]:
import pickle

In [52]:
with open("tfidf.pkl", "wb") as f:
    pickle.dump(tfid, f)

In [54]:
with open("rf.pkl", "wb") as f:
    pickle.dump(rf1, f)

In [ ]:
# using the transformers

!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00


In [ ]:

from transformers import BertModel, BertTokenizer

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import torch

bert_emb = []
# bert_emb.shape

for index, row in enumerate(X_train.itertuples()):

  input_text = row.transformed_tweet
  input_ids = tokenizer.encode(input_text, add_special_tokens=True)
  input_ids = torch.tensor([input_ids])

  with torch.no_grad():
      last_hidden_states = model(input_ids)[0] # Models outputs are now tuples
  last_hidden_states = last_hidden_states.mean(1)
  # print(last_hidden_states)
  print(type(last_hidden_states))
  bert_emb.extend(np.array(last_hidden_states.tolist()))




In [ ]:
combined_data1 = pd.concat([pd.DataFrame(bert_emb),
                           pd.DataFrame( X_ext_feat)], axis=1,ignore_index=True)

In [ ]:
combined_data1.to_csv("bert_emb_train.csv", index=False)

In [ ]:
combined_data1


,0,1,2,3,4,5,6,7,8,9,...,763,764,765,766,767,768,769,770,771,772
0,-0.087279,0.230925,0.577131,-0.120769,-0.056001,-0.056415,0.521712,0.719100,-0.370716,-0.201459,...,-0.274699,-0.110190,-0.426164,0.098789,-0.315646,0.148601,0.182927,0.000,0.494,0.500000
1,-0.252210,0.222189,0.283385,0.026770,-0.051060,-0.190175,0.032601,-0.174828,-0.151958,-0.461344,...,0.005514,-0.271940,-0.399949,0.222155,0.094684,0.055944,0.048780,0.429,0.000,0.750000
2,0.024303,0.019568,0.655474,-0.196554,0.173366,-0.153032,0.774453,0.294535,-0.163257,-0.160735,...,-0.033764,0.127240,-0.361043,0.102529,-0.138611,0.139860,0.182927,0.115,0.332,0.250000
3,0.148752,0.033611,0.422663,-0.025314,0.042489,0.043177,0.207770,0.332749,-0.091647,-0.264388,...,-0.230680,-0.089675,-0.514033,0.256302,-0.261781,0.050699,0.048780,0.000,0.000,0.500000
4,0.559337,0.172149,0.486911,0.018039,0.251186,-0.135627,0.415722,0.498320,-0.417695,-0.048705,...,0.044901,-0.113433,-0.583305,-0.073075,-0.492028,0.173077,0.182927,0.258,0.266,0.487500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17958,0.531728,-0.118953,0.596241,-0.119564,0.119456,-0.089948,0.298877,0.155741,-0.406554,-0.037960,...,0.326506,-0.275196,-0.201105,-0.125049,-0.066417,0.139860,0.170732,0.198,0.125,0.900000
17959,0.250856,0.060807,0.479395,-0.212602,0.094050,-0.088139,0.438165,0.245734,-0.039321,-0.303516,...,0.012045,-0.055296,-0.361495,-0.385379,-0.037832,0.097902,0.097561,0.277,0.000,0.500000
17960,0.533168,0.265795,0.454215,-0.137734,0.127945,-0.068889,0.624046,0.103537,-0.272618,-0.454007,...,0.341567,0.181359,-0.347618,0.064388,-0.016040,0.099650,0.121951,0.165,0.485,0.321875
17961,-0.149179,-0.028914,0.590737,-0.026689,-0.473292,-0.072776,0.450900,0.414378,-0.165446,-0.390988,...,-0.047443,0.257967,-0.236880,-0.112283,0.161029,0.041958,0.048780,0.000,0.487,0.500000


In [ ]:

bert_emb_test = []
# bert_emb.shape

for index, row in enumerate(X_test.itertuples()):


  input_text = row.transformed_tweet
  input_ids = tokenizer.encode(input_text, add_special_tokens=True)
  input_ids = torch.tensor([input_ids])

  with torch.no_grad():
      last_hidden_states = model(input_ids)[0]
  last_hidden_states = last_hidden_states.mean(1)
  # print(last_hidden_states)
  print(type(last_hidden_states))
  bert_emb_test.extend(np.array(last_hidden_states.tolist()))


In [ ]:
X_ext_feat.shape


(17963, 5)

In [ ]:
combined_data_test1 = pd.concat([pd.DataFrame(bert_emb_test),
                           pd.DataFrame(X_test_ext_feat)], axis=1,ignore_index=True)

In [ ]:
combined_data_test1
type(combined_data_test1)

pandas.core.frame.DataFrame

In [ ]:
combined_data_test1.to_csv("bert_emb__test.csv", index=False)


In [ ]:
combined_data_test1

,0,1,2,3,4,5,6,7,8,9,...,763,764,765,766,767,768,769,770,771,772
0,0.279496,-0.048461,0.329765,0.081726,0.294171,-0.142551,0.256896,0.466527,-0.037355,-0.310173,...,-0.237634,-0.187456,-0.268861,0.040591,0.096857,0.193548,0.163265,0.000000,0.000,0.500000
1,0.285358,0.331505,0.175770,-0.018891,-0.184914,0.148873,0.636299,0.200386,-0.039707,-0.104912,...,-0.177869,0.118157,-0.576273,-0.035299,-0.360115,0.108504,0.102041,0.000000,0.612,0.350000
2,-0.145431,0.388222,0.818351,0.041443,0.422156,0.059698,0.245908,0.278290,-0.112323,-0.109106,...,0.042214,0.044899,-0.299049,0.027567,-0.078617,0.152493,0.142857,0.178005,0.000,0.500000
3,0.262193,0.157569,0.732446,-0.355400,-0.017819,0.080225,0.478646,0.234520,-0.372133,-0.036313,...,0.100350,0.162992,-0.479646,-0.083244,0.017884,0.258065,0.306122,0.310658,0.459,0.575000
4,0.266512,0.032934,0.319085,-0.290127,-0.111751,0.198891,0.457307,0.388211,-0.349824,-0.132010,...,-0.103924,-0.051250,-0.245271,-0.158707,-0.250674,0.090909,0.122449,0.000000,0.597,0.275000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4486,0.057983,0.128221,0.874461,-0.244329,0.132839,0.096809,-0.100599,0.369570,-0.147157,-0.271978,...,0.243485,-0.371262,0.100264,-0.091118,0.449397,0.058651,0.040816,0.000000,0.000,0.500000
4487,0.534971,-0.267877,1.006449,-0.183908,0.246927,-0.073014,0.268915,0.359940,-0.326323,-0.069395,...,-0.017884,-0.014271,-0.565289,0.085335,-0.567206,0.126100,0.142857,0.531746,0.259,0.800000
4488,0.250200,-0.024549,0.777601,-0.424235,-0.196184,0.158928,0.516017,0.196120,-0.347829,0.048024,...,-0.415489,-0.243774,-0.498644,0.003460,-0.090554,0.217009,0.244898,0.140590,0.378,0.400000
4489,0.134157,-0.061019,0.534790,-0.037469,-0.108869,-0.068058,0.390058,-0.361031,-0.080204,-0.329018,...,0.323149,0.018406,-0.159382,0.210896,-0.196529,0.052786,0.040816,0.000000,0.655,0.500000


In [ ]:
gn.fit(combined_data1,y_train)
pred1=gn.predict(combined_data_test1)
print(pred1)
print(accuracy_score(y_test,pred1))
print(confusion_matrix(y_test,pred1))
print(precision_score(y_test,pred1))
print(classification_report(y_test, pred1))

[1 0 0 ... 0 0 0]
0.7833444667112002
[[2935  852]
 [ 121  583]]
0.4062717770034843
              precision    recall  f1-score   support

           0       0.96      0.78      0.86      3787
           1       0.41      0.83      0.55       704

    accuracy                           0.78      4491
   macro avg       0.68      0.80      0.70      4491
weighted avg       0.87      0.78      0.81      4491



In [ ]:
rf=RandomForestClassifier()

rf.fit(combined_data1,y_train)
pred11=rf.predict(combined_data_test1)
print(pred11)
print(accuracy_score(y_test,pred11))
print(confusion_matrix(y_test,pred11))
print(precision_score(y_test,pred11))
print(classification_report(y_test, pred11))

[0 0 0 ... 0 0 0]
0.8842128701848141
[[3724   63]
 [ 457  247]]
0.7967741935483871
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      3787
           1       0.80      0.35      0.49       704

    accuracy                           0.88      4491
   macro avg       0.84      0.67      0.71      4491
weighted avg       0.88      0.88      0.86      4491



In [ ]:
bn.fit(combined_data1,y_train)
pred2=bn.predict(combined_data_test1)
print(pred2)
print(accuracy_score(y_test,pred2))
print(confusion_matrix(y_test,pred2))
print(precision_score(y_test,pred2))
print(classification_report(y_test, pred2))

[1 0 0 ... 0 0 0]
0.796481852594077
[[3026  761]
 [ 153  551]]
0.41996951219512196
              precision    recall  f1-score   support

           0       0.95      0.80      0.87      3787
           1       0.42      0.78      0.55       704

    accuracy                           0.80      4491
   macro avg       0.69      0.79      0.71      4491
weighted avg       0.87      0.80      0.82      4491



In [ ]:
lr=LogisticRegression()
lr.fit(combined_data1,y_train)
pred_lr=lr.predict(combined_data_test1)

print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3787
           1       0.71      0.71      0.71       704

    accuracy                           0.91      4491
   macro avg       0.83      0.83      0.83      4491
weighted avg       0.91      0.91      0.91      4491



In [ ]:
print(confusion_matrix(y_test,pred_lr))

[[3585  202]
 [ 206  498]]


In [ ]:
xg = XGBClassifier()
xg.fit(combined_data1, y_train)
pred_xg=xg.predict(combined_data_test1)

print(classification_report(y_test, pred_xg))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      3787
           1       0.74      0.58      0.65       704

    accuracy                           0.90      4491
   macro avg       0.83      0.77      0.80      4491
weighted avg       0.90      0.90      0.90      4491



In [ ]:
print(confusion_matrix(y_test,pred_xg))

[[3641  146]
 [ 295  409]]
